# Combustion Chemistry in Isobaric Reactor: Generatign Training Database in the Principal Component Space

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import time


### Defining WORKSPACE_PATH

# WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.path.join(os.getcwd(), '../../../../../')
ROMNet_fld     = os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/')


### Importing External Libraries

import numpy                             as np
import pandas                            as pd
pd.options.mode.chained_assignment = None  # default='warn'


### Importing Matplotlib and Its Style

import matplotlib.pyplot                 as plt

#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/presentation.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/zoomed.mplstyle'))
plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_1column.mplstyle'))
#plt.style.use(os.path.join(WORKSPACE_PATH, 'ROMNet/romnet/extra/postprocessing/paper_2columns.mplstyle'))


# from scipy.integrate import solve_ivp
# import pyDOE    
# from PCAfold         import PCA          as PCAA

3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]


Missing colon in file PosixPath('/Users/sventur/.matplotlib/stylelib/paper_2columns.mplstyle'), line 1 ('f#### MATPLOTLIBRC FORMAT')


## Defining Input Variables

In [2]:
OutputDir    = os.path.join(WORKSPACE_PATH, 'ROMNet/Data/0DReact_Isobaric_1000Cases_CH4/')
WriteDir     = OutputDir + '/Autoencoder/'

valid_perc   = 20.

FixedMinVal  = 1.e-14

n_ics        = 1000


ColorVec     = ['#190707', '#dd3232', '#0065a9', '#348a00','#985396','#f68b69']

try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

In [3]:
FileName    = OutputDir+'/Orig/train/ext/CleanVars.csv'
input_vars  = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()[0,:]

FileName    = OutputDir+'/Orig/train/ext/CleanVars.csv'
output_vars = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()[0,:]

In [4]:
j_ics        = 0
for i_ics in range(n_ics):
    FileName = OutputDir + '/Orig/train/ext/y.csv.'+str(i_ics+1)
    if (j_ics == 0):
        output_data = pd.read_csv(FileName)
    else:
        output_data = pd.concat([output_data, pd.read_csv(FileName)], ignore_index=True)
    j_ics  += 1
output_data = output_data[output_vars]

input_data  = output_data[input_vars]

In [5]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(WriteDir+'/train/')
except:
    pass
try:
    os.makedirs(WriteDir+'/valid/')
except:
    pass

In [6]:
data_id              = 'pts'

try:
    os.makedirs(WriteDir+'/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(WriteDir+'/valid/'+data_id+'/')
except:
    pass


DataOutput = output_data.copy()
# numerics   = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# for c in [c for c in DataOutput.columns if DataOutput[c].dtype in numerics]:
#     DataOutput[c] = np.log10(DataOutput[c] + 1.e-20) + 20.
Idxs       = DataOutput.isna().any(axis=1) == True
DataOutput.drop(DataOutput[Idxs].index, inplace=True)

DataInput  = input_data.copy()
DataInput.drop(DataInput[Idxs].index, inplace=True)



n_points             = len(DataOutput)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=valid_perc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)


DataInput.iloc[train_idx].to_csv(WriteDir+'/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(WriteDir+'/valid/'+data_id+'/Input.csv', index=False)

DataOutput.iloc[train_idx].to_csv(WriteDir+'/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(WriteDir+'/valid/'+data_id+'/Output.csv', index=False)